In [ ]:
#@title Step 1: Importing the necessary libraries
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pickle
import time
import itertools
import os

In [ ]:
#@title Step 2: Load preprocessed (sequenced) data; Splitting data into training and testing datasets

import pickle

# Load sequences and vehicle IDs from a file
with open('sequences_vehicle_ids.pkl', 'rb') as f:
#with open('sequences_vehicle_ids.pkl', 'rb') as f:
    sequences, vehicle_ids = pickle.load(f)

# Split data into training and testing
train_size = int(0.8 * len(sequences))
train_sequences = sequences[:train_size]
test_sequences = sequences[train_size:]
train_vehicle_ids = vehicle_ids[:train_size]
test_vehicle_ids = vehicle_ids[train_size:]

class VehicleDataset(Dataset):
    def __init__(self, sequences, flatten=False):
        self.sequences = sequences
        self.flatten = flatten # new

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq, target = self.sequences[idx]
        seq_tensor = torch.Tensor(seq)
        if self.flatten:
            seq_tensor = seq_tensor.view(-1)  # Flatten the sequence
        return seq_tensor, torch.Tensor(target)

# Create DataLoader objects for the full dataset
train_dataset_all = VehicleDataset(train_sequences, flatten=False)
test_dataset_all = VehicleDataset(test_sequences, flatten=False)

train_loader_all = DataLoader(train_dataset_all, batch_size=32, shuffle=True)
test_loader_all = DataLoader(test_dataset_all, batch_size=32, shuffle=False)

# Split train dataset for hyperparameter tuning
tune_size = int(0.2 * len(train_sequences))  # Use 10% of the training data for tuning
tune_sequences = train_sequences[:tune_size]
tune_vehicle_ids = train_vehicle_ids[:tune_size]

tune_dataset = VehicleDataset(tune_sequences, flatten=False)
tune_test_size = int(0.2 * len(test_sequences))  # Use 10% of the testing data for tuning
tune_test_sequences = test_sequences[:tune_test_size]

tune_test_dataset = VehicleDataset(tune_test_sequences, flatten=False)

# Create DataLoader objects for tuning set
train_loader = DataLoader(tune_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(tune_test_dataset, batch_size=32, shuffle=False)

In [ ]:
#@title Step 3: Defining the methods (architecture of NN layers and activation functions)

#### Step 3a: Define the Long Short-Term Memory (LSTM) model

import torch.nn as nn
import torch.nn.functional as F

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

#### Step 3b: Define the Gated Recurrent Unit (GRU) model

class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])
        return out


#### Step 3c: Define the generic Recurrent Neural Network (RNN) model

class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out


#### Step 3d: Define 1-d CNN (Conv1D) model

class Conv1DModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(Conv1DModel, self).__init__()
        self.conv_layers = nn.ModuleList()
        self.pool_layers = nn.ModuleList()

        # Input layer
        self.conv_layers.append(nn.Conv1d(input_size, hidden_size, kernel_size=3, padding=1))
        self.pool_layers.append(nn.MaxPool1d(kernel_size=2, stride=1))  # Reduced stride

        # Hidden layers
        for _ in range(1, num_layers):
            self.conv_layers.append(nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1))
            self.pool_layers.append(nn.MaxPool1d(kernel_size=2, stride=1))  # Reduced stride

        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.transpose(1, 2)  # Swap dimensions to fit Conv1d input format
        for conv, pool in zip(self.conv_layers, self.pool_layers):
            x = F.relu(conv(x))
            x = pool(x)
        x = x.mean(dim=2)  # Global average pooling
        x = self.fc(x)
        return x
    
#### Step 3e: Define a simple Multi-layer Perceptron (MLP) model

class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(MLPModel, self).__init__()
        layers = [nn.Linear(input_size, hidden_size), nn.ReLU()]

        for _ in range(1, num_layers):
            layers.extend([nn.Linear(hidden_size, hidden_size), nn.ReLU()])

        layers.append(nn.Linear(hidden_size, output_size))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x.view(x.size(0), -1))

In [ ]:
#@title Step 4: Training method (generic for all baselines)

def train_model(model, train_loader, criterion, optimizer, num_epochs=10, device='cuda', early_stopping_rounds=4, min_delta=0.01):
    model.to(device)
    epoch_times = []
    epoch_losses = []

    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()
        total_loss = 0
        for sequences, targets in train_loader:
            sequences, targets = sequences.to(device), targets.to(device)
            outputs = model(sequences)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        average_loss = total_loss / len(train_loader)
        epoch_time = time.time() - start_time
        epoch_times.append(epoch_time)
        epoch_losses.append(average_loss)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.8f}, Time: {epoch_time:.2f}s')

        # Early stopping mechanism
        if epoch >= early_stopping_rounds:
          recent_losses = epoch_losses[-early_stopping_rounds:]
          if all(abs(recent_losses[i] - recent_losses[i-1]) < min_delta * recent_losses[i-1] for i in range(1, early_stopping_rounds)):
              print(f"Early stopping at epoch {epoch+1}")
              break

    return epoch_times, epoch_losses

In [ ]:
#@title Step 5: Model Evaluation

from sklearn.metrics import mean_absolute_error, r2_score

def evaluate_model(model, test_loader, device='cuda'):
    # device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)

    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for sequences, targets in test_loader:
            sequences, targets = sequences.to(device), targets.to(device)
            outputs = model(sequences)
            predictions.extend(outputs.cpu().numpy())
            actuals.extend(targets.cpu().numpy())

    predictions = np.array(predictions)
    actuals = np.array(actuals)

    rmse = np.sqrt(np.mean((predictions - actuals) ** 2))
    mae = mean_absolute_error(actuals, predictions)
    r2 = r2_score(actuals, predictions)

    print(f'RMSE: {rmse:.8f}')
    print(f'MAE: {mae:.8f}')
    print(f'R2 Score: {r2:.8f}')

    return predictions, actuals, rmse, mae, r2

In [ ]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU available")
print(torch.__version__)
print(torch.version.cuda)


In [ ]:
#@title Step 6: Define a method to run the training and evaluation of the model for different sets of hyperparameters

def run_experiment(params, input_size=4, output_size=2, model_type='LSTM'):
    hidden_size, num_layers, learning_rate, num_epochs = params

    if model_type == 'LSTM':
        model = LSTMModel(input_size, hidden_size, num_layers, output_size)
        save_path = 'experiment_results_LSTM.csv'
    elif model_type == 'GRU':
        model = GRUModel(input_size, hidden_size, num_layers, output_size)
        save_path = 'experiment_results_GRU.csv'
    elif model_type == 'RNN':
        model = RNNModel(input_size, hidden_size, num_layers, output_size)
        save_path = 'experiment_results_RNN.csv'
    elif model_type == 'Conv1D':
        model = Conv1DModel(input_size, hidden_size, num_layers, output_size)
        save_path = 'experiment_results_Conv1D.csv'
    elif model_type == 'MLP':
        example_input, _ = next(iter(train_loader))
        flattened_input_size = example_input.view(example_input.size(0), -1).size(1)
        model = MLPModel(flattened_input_size, hidden_size, num_layers, output_size)
        save_path = 'experiment_results_MLP.csv'
    else:
        raise ValueError(f"Unsupported model type: {model_type}")

    # Initialize criterion and optimizer
    criterion = nn.MSELoss() # mean squared error (MSE) is the criterion for evaluating loss in each epoch
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # Using Adam optimizer

    # Train the model and measure time
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Use the provided train_model function
    epoch_times, epoch_losses = train_model(model, train_loader, criterion, optimizer, num_epochs=num_epochs, device=device)

    # Evaluate the model
    predictions, actuals, rmse, mae, r2 = evaluate_model(model, test_loader, device=device)

    # Calculate average epoch time
    avg_epoch_time = np.mean(epoch_times)

    result = {
        'hidden_size': hidden_size,
        'num_layers': num_layers,
        'learning_rate': learning_rate,
        'num_epochs': num_epochs,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'avg_epoch_time': avg_epoch_time,
        'epoch_losses': epoch_losses
    }

    results_df = pd.DataFrame([result])
    if save_path:
        if os.path.exists(save_path):
            results_df.to_csv(save_path, mode='a', header=False, index=False)
        else:
            results_df.to_csv(save_path, index=False)

    return result


In [ ]:
#@title Step 7: Define hyperparameter sets

# Define the hyperparameter grid (2*3*8*1 = 48 total possibilities)
param_grid = {
    'hidden_size': [50, 100],
    'num_layers': [2, 3, 4],
    'learning_rate': [0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005],
    'num_epochs': [30] 
}

# Create a list of hyperparameter combinations
param_combinations = list(itertools.product(
    param_grid['hidden_size'],
    param_grid['num_layers'],
    param_grid['learning_rate'],
    param_grid['num_epochs']
))

def run_specific_experiment(index, input_size=4, output_size=2, model_type='LSTM'): # allowed number for index: 0-47
    if index < 0 or index >= len(param_combinations):
        raise ValueError("Index out of range")
    
    params = param_combinations[index]

    print('model type: ', model_type)
    print('index of experiment / total number of experiments: ', str(index+1), ' / ',str(len(param_combinations)))
    print('Hyperparameters set (hidden_size, num_layers, learning_rate, num_epochs): ', params)

    result = run_experiment(params, input_size, output_size, model_type)
    
    return result

In [ ]:
#@title Step 8: Execute each method (flexible for continuing unfinished experiments)

# We have 48 sets of experiments due to the different values for hyperparameters in param_grid (defined in Step 7)
for i in range(48):
    result = run_specific_experiment(index=i, model_type='LSTM')
    print('---------')
for i in range(48):
    result = run_specific_experiment(index=i, model_type='GRU')
    print('---------')
for i in range(48):
    result = run_specific_experiment(index=i, model_type='RNN')
    print('---------')
for i in range(48): 
    result = run_specific_experiment(index=i, model_type='Conv1D')
    print('---------')
for i in range(48):
    result = run_specific_experiment(index=i, model_type='MLP')
    print('---------')

In [ ]:
#@title Step 9: Visualize Results to evalate different hyperparameters
import matplotlib.pyplot as plt

save_path = 'experiment_results_LSTM.csv' # change LSTM with other methods to visualize metrics for those methods (Methods: RNN, GRU, LSTM, Conv1D, MLP)
results_df = pd.read_csv(save_path)


# Plot RMSE for different hyperparameters
plt.figure(figsize=(14, 7))
for key, grp in results_df.groupby(['hidden_size', 'num_layers']):
    plt.plot(grp['learning_rate'], grp['rmse'], label=f"hidden_size={key[0]}, num_layers={key[1]}")
plt.xlabel('Learning Rate')
plt.ylabel('RMSE')
plt.xscale('log')
plt.title('RMSE for Different Hyperparameter Combinations')
plt.legend()
plt.show()

# Plot MAE for different hyperparameters
plt.figure(figsize=(14, 7))
for key, grp in results_df.groupby(['hidden_size', 'num_layers']):
    plt.plot(grp['learning_rate'], grp['mae'], label=f"hidden_size={key[0]}, num_layers={key[1]}")
plt.xlabel('Learning Rate')
plt.ylabel('MAE')
plt.xscale('log')
plt.title('MAE for Different Hyperparameter Combinations')
plt.legend()
plt.show()

# Plot R2 Score for different hyperparameters
plt.figure(figsize=(14, 7))
for key, grp in results_df.groupby(['hidden_size', 'num_layers']):
    plt.plot(grp['learning_rate'], grp['r2'], label=f"hidden_size={key[0]}, num_layers={key[1]}")
plt.xlabel('Learning Rate')
plt.ylabel('R2 Score')
plt.xscale('log')
plt.title('R2 Score for Different Hyperparameter Combinations')
plt.legend()
plt.show()


# Plot loss over epochs for different hyperparameters
plt.figure(figsize=(14, 7))
for idx, result in results_df.iterrows():
    epoch_losses = eval(result['epoch_losses'])  # Convert string representation of list back to list
    plt.plot(epoch_losses, label=f"hidden_size={result['hidden_size']}, num_layers={result['num_layers']}, lr={result['learning_rate']}, epochs={result['num_epochs']}")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs for Different Hyperparameter Combinations')
plt.legend()
plt.show()
